<a href="https://colab.research.google.com/github/gastonbujia/curso-visualizacion-datos-SAN-2022/blob/main/notebooks/actividades/comportamientoCalms21_Ejercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>
CalMS21: Caltech Mouse Social Interaction Dataset 2021 🐁🐀
</h1>

Esta notebook es una adaptación de la provista por NeuroMatch Academy durante la escuela de [Computational Neuroscience](https://compneuro.neuromatch.io/tutorials/intro.html) curada por Ann Kenedy, que a su vez es una adapatación de la que fuera creada por Dipam Chakraborty y Sharada Mohanty de AIcrowd para el <a href=https://www.aicrowd.com/challenges/multi-agent-behavior-representation-modeling-measurement-and-applications>Multi-Agent Behavior Challenge</a>. La notebook original puede ser encontrada en este [link](https://compneuro.neuromatch.io/projects/behavior/README.html).

Este [dataset](https://data.caltech.edu/records/s0vdx-0k302) tiene estimaciones de pose-tracking de ratones interactuando socialmente mediante capturas de video y además cuenta con anotaciones sobre 3 posibles interacciones sociales: *investigacion*, *montar* y *atacar* separada de *otros* compartamientos.


# Imports e instalaciones 📚


In [ ]:
import os
import json
import numpy as np
import pandas as pd

# Algunas librerias para graficar
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Seteamos el estilo
sns.set(style="darkgrid")

# Reducción de la dimensión y otros
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Dataset CalMS21 📲

El dataset **CalMS21** esta hosteado y mantenido por Caltech en https://data.caltech.edu/records/1991. Brevemente, CalMS21 contiene videos y datos anotados de una tarea experimental en la cual se estudia el comportamiento social entre dos ratones, uno que es residente del ambiente y un segundo que es introducido en el mismo. Las interacciones duran aproximadamente unos 10 minutos, son grabadas en un plano cenital y se anotan los comportamientos registrados durante ese tiempo. De los videos se extraen 7 puntos (`keypoints`) para cada ratón estimados por un red neuronal y una confianza de la localización de estos puntos.

![](https://images.aicrowd.com/uploads/ckeditor/pictures/332/content_expt_setup_merged.gif)

Pueden encontrar [acá](https://www.youtube.com/watch?v=tDmhmasjPeM) un video donde una de las autoras del trabajo nos cuenta sobre este dataset.

Dado que el dataset fue construido con el objetivo de probar técnicas de Machine Learning y Deep Learning, hay más de una tarea expermiental, y cada una de ellas contará con un conjunto de *entrenamiento*(`train`) y de *evaluación*(`test`). En esta notebook solo usaremos los datos anotados como la *tarea 1* (`task1`) y el conjunto de entrenamiento. 

In [ ]:
# @title Descarga y unzip
import os, requests, zipfile

# Descargamos los datos a colab
fname = 'task1_classic_classification.zip'
url = "https://data.caltech.edu/records/s0vdx-0k302/files/task1_classic_classification.zip?download=1"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)
else:
  print('Data have already been downloaded!!!')

if not os.path.exists('task1_classic_classification'):
  # Unzip the file
  with zipfile.ZipFile(fname, 'r') as zip_ref:
    zip_ref.extractall('.')

# Descargamos el parser
fname = 'calms21_convert_to_npy.py'
url = "https://data.caltech.edu/records/s0vdx-0k302/files/calms21_convert_to_npy.py?download=1"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

Los archivos del conjunto de datos se almacenan como archivos json. Para facilitar el manejo, primero los convertiremos a archivos .npy usando el script que acabamos de descargar, `calms21_convert_to_npy.py`. La salida de este script es un par de archivos, `calms21_task1_train.npy` y `calms21_task1_test.npy`. Como nosotros no vamos a intentar hacer predicciones con estos datos sino un análisis exploratorio.

El repositorio cuenta con un script `calms21_convert_to_npy.py` que permite convertir los datos que estan en formato `JSON` a un formato que nos permite leerlo más facilmente: NumPy Array. En la próxima celda mostramos como utilizar el script pasandole los directorios donde se almacenaron los datos descargados. Si se incluye el indicador `parse_treba` opcional, el script creará los archivos `calms21_task1_train_features.npy` y `calms21_task1_test_features.npy`, que contienen unos 60 *features* creadas con <a href=https://openaccess.thecvf.com/content/CVPR2021/html/Sun_Task_Programming_Learning_Data_Efficient_Behavior_Representations_CVPR_2021_paper.html>Task Programming</a>. Mas adelante también exploraremos esta información.



In [ ]:
# @title Ejecutar celda para extraer y crear los archivos necesarios
!python calms21_convert_to_npy.py  --input_directory '.' --output_directory '.'
!python calms21_convert_to_npy.py  --input_directory '.' --output_directory '.' --parse_treba

# Cargamos la data 💾
A continuación proveemos una función que permite cargar los datos. Si quieren exploren y traten de entender que es lo que hace.

In [ ]:
def load_task1_data(data_path, features_path):
  """
  Cargar datos para la tarea 1:
       El vocabulario le dice cómo asignar nombres de comportamiento a identificadores de clase;
       es el mismo para todas las secuencias en este conjunto de datos.
  """
  data_dict = np.load(data_path, allow_pickle=True).item()
  dataset = data_dict['annotator-id_0']
  # Obtener una secuencia y sus llaves.
  sequence_id = list(data_dict['annotator-id_0'].keys())[0]
  vocabulary = data_dict['annotator-id_0'][sequence_id]['metadata']['vocab']
  features = np.load(features_path, allow_pickle=True).item()['annotator-id_0']
  return dataset, features, vocabulary

In [ ]:
training_data, training_features, vocab = load_task1_data('./calms21_task1_train.npy', './calms21_task1_train_features.npy')

## Dataset

Las variables recién creadas `training_data` y `test_data` son diccionarios con una clave para cada **Secuencia** en el conjunto de datos, donde una **Secuencia** es un único ensayo residente-intruso, *hay un total de 70 ensayos*. Cada **Secuencia** contiene los siguientes campos:

<ul>
<li><b>keypoints</b>: ubicaciones rastreadas de partes del cuerpo en los dos ratones que interactúan. Estos se producen utilizando una red de reloj de arena apilado entrenada en 15,000 marcos etiquetados a mano.
<ul>
<li>Dimensiones: (n.º de fotogramas) x (ID del ratón) x (coordenada x, y) x (parte del cuerpo).
<li>Unidades: píxeles; las coordenadas son relativas a la imagen completa. Las dimensiones de la imagen original son 1024 x 570.
</ul>
<li><b>scores</b>: estimaciones de confianza para los puntos clave rastreados.
<ul>
<li>Dimensiones: (n.º de fotogramas) x (ID del ratón) x (parte del cuerpo).
<li>Unidades: sin unidades, rango de 0 (confianza más baja) a 1 (confianza más alta).
</ul>
<li> <b>annotations</b>: ID de comportamientos como un número entero anotado en cada cuadro por un experto en el dominio. Consulte a continuación las asignaciones de ID de comportamiento a nombre de comportamiento.
<ul>
<li>Dimensiones: (# cuadros) .
</ul>
<li><b>metadata</b>: los metadatos registrados son annotator_id, que se representa mediante un int, y el vocabulario, que contiene un diccionario que asigna nombres de comportamiento a identificadores enteros en las anotaciones.
</ul>

El archivo 'taskprog_features' contiene el campo adicional:

<ul>
<li><b>features</b>: características precalculadas de un modelo entrenado con programación de tareas en los datos de trayectoria del conjunto de videos sin etiqueta CalMS21.
<ul>
<li>Dimensiones: (n.º de fotogramas) x (dimensión característica = 32).
</li>
</ul>
</ul>

![](https://images.aicrowd.com/uploads/ckeditor/pictures/337/content_keypoints_task_1.png)

<b>NOTA:</b> para todos los puntos clave, el ratón 0 es el ratón residente (negro) y el ratón 1 es el ratón intruso (blanco). Hay 7 partes del cuerpo rastreadas, ordenadas (nose, left ear, right ear, neck, left hip, right hip, tail base).

## ¿Cómo se ven los datos? 🔍

### Resumen de datos

Como se describió anteriormente, nuestro conjunto de datos consta de diccionarios de Secuencias y un vocabulario adjunto que nos dice qué comportamiento es resgistrado con que código numérico:

In [ ]:
print("Algunas keys: ", list(training_data.keys())[:3])
print("Vocabulario: ", vocab)
print("Numero de secuencias: ", len(training_data))

In [ ]:
# Veamos en que formato esta y que tipos tienen para ver si podemos cargarlo facilmente a Pandas
print(f"Tipo de training data: {type(training_data)}")
print(f"Tipo de cada dato de un ensayo: {type(training_data['task1/train/mouse001_task1_annotator1'])}")
sequence_names = list(training_data.keys())
sample_sequence_key = sequence_names[0]
single_sequence = training_data[sample_sequence_key]
print("Nombre de nuestra secuencia: ", sample_sequence_key)
print("Keys de la secuencia: ", single_sequence.keys())
print("Metadata: ", single_sequence['metadata'])
print(f"Numero de frames en una secuencia \"{sample_sequence_key}\": ", len(single_sequence['annotations']))
print(f"Tamaño de los keypoints de una secuencia \"{sample_sequence_key}\": ", single_sequence['keypoints'].shape)

No parece ser tan fácil cargarlo directamente en un DataFrame de Pandas, pero pueden intentarlo en la próxima celda a ver que sucede...

In [ ]:
# COMPLETAR pasarlo directamente a dataframe e imprimirlo
df = ...
df

¿Tiene sentido cargarlo de esta manera? ¿Que representa cada una de las dimensiones de dataframe cargado de esta manera?

# Ejercicio 1: Explorando narices 👃

Hay varias preguntas que nos pueden interesar a la hora de estudiar este conjunto de datos que tiene que ver con describir las posiciones del cuerpo de los ratones según cada comportamiento. Para este ejercicio nos va a interesar explorar que pasa con las narices de los ratones en cada comportamiento para el primer ensayo.

## 1.1 Nos vamos para Pandas! 🐼

a) Como vimos recién vamos a tener que trabajar un poco el dataset para poder cargarlo en Pandas, por eso primero tienen que completar la función de abajo que procesa uno de los ensayos y nos devuelve toda la información en un unico dataframe. *Hint:* Mirar la función `pd.concat` 

In [ ]:
# COMPLETAR esta función para que procese el diccionario de anotaciones en un solo dataframe donde cada columna va a ser una coordenada de un keypoint de cada raton y la confianza
def load_annotator_df(data_dict, annotator):
    
    # vamos a trabajar solo con un ensayo
    dict_aux = data_dict[annotator]
    # vocabulario
    annot_keys = {0:'attack',1: 'investigation',2: 'mount',3:'other'}
    # nombre de los keypoints
    body_parts = ["nose", "left_ear", "right_ear", "neck", "left_hip", "right_hip", "tail_base"]
    # una lista donde iremos dejando cada dataframe para luego unirlos en un unico dataframe
    df_list = []
    
    # desagregamos los keypoints en columnas para cada parte trackeada / aclarar aca que podemos tambien usar una lista por comprension / reemplazar por lista
    df_list.append(pd.DataFrame(dict_aux['keypoints'][:,0,0,:] , dtype=float, columns=['keypoints_mousse_resident_x_'+i for i in body_parts]))
    df_list.append(pd.DataFrame(dict_aux['keypoints'][:,0,1,:] , dtype=float, columns=['keypoints_mousse_resident_y_'+i for i in body_parts]))
    df_list.append(pd.DataFrame(dict_aux['keypoints'][:,1,0,:] , dtype=float, columns=['keypoints_mousse_intruder_x_'+i for i in body_parts]))
    df_list.append(pd.DataFrame(dict_aux['keypoints'][:,1,1,:] , dtype=float, columns=['keypoints_mousse_intruder_y_'+i for i in body_parts]))
    
    # desagregamos la confianza de los keypoints 
    df_list.append(pd.DataFrame(dict_aux['scores'][:,0,:] , dtype=float, columns=['keypoints_mousse_resident_confidence_'+i for i in body_parts]))
    df_list.append(pd.DataFrame(dict_aux['scores'][:,1,:] , dtype=float, columns=['keypoints_mousse_intruder_confidence_'+i for i in body_parts]))

    # COMPLETAR: crear un único dataframe que sea el resultado de concatenar todos los dataframes de la lista df_list que contenga todas las columnas
    df = ...

    # COMPLETAR: agregar al dataframe recien creado una columna con la etiqueta correspondiente
    df['annotations'] = ...

    # COMPLETAR: reemplazar la columna annotations por una que codifique el comportamiento anotado en el vocabulario (miren la funcin map de pandas)
    df['annotations'] = ...
    
    return df

In [ ]:
df = load_annotator_df(training_data, 'task1/train/mouse001_task1_annotator1')
df.head()

b) ¿Que comportamientos se registraron para este ensayo?

In [ ]:
# COMPLETAR Imprimir los valores unicos de la columna annotations
...

## 1.2 Nuevas columnas

Vamos a crear algunas columnas a partir del dataframe cargado que nos permitan caracterizar los comportamientos encontrados en este ensayo y nos basaremos en las distancais de las narices!


a) Para esto queremos mirar los frames que esten bien estimados, vamos a quedarnos solo con aquellos que tienen una estimación de la nariz más alta que 0.5. Primero seleccionen y grafiquen la distribución de esa confianza para las narices para ambos ratones con un histograma conjunto (o pueden usar dos). *Hint* Pueden usar el parámetro `alpha` que controla la transparencia de las barras.

In [ ]:
# COMPLETAR: Graficar la distribución de la confianza de los keypoints estimate para la nariz de cada raton
...

b) Filtren el dataset en base a que ambas columnas sean mayor al corte `thres`. *Hint* Al tener mas de una condición, dependiendo de que quieran hacer las puede servir el comando [`all`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.all.html)

In [ ]:
# COMPLETAR Crear una mascara con el punto de corte y crear un nuevo dataset a partir de ese
cols = ["keypoints_mousse_resident_confidence_nose", "keypoints_mousse_intruder_confidence_nose"]
thres = 0.5
df_filter = ...

# PREGUNTA: Cuantos datos tiraron?
total_frames  = df.shape[0]
total_tirados = ...
print(f'Descartamos un total del {...}% de frames')

c) Ahora si podemos pensar en crear la columna que mida la distancia entre las narices de ambos ratones. Para eso podemos calcular la distancia euclídea entre dos puntos de coordenadas $(x_0,y_0)$ y $(x_1,y_1)$ como:

$d=\sqrt{(x_1-x_0)^2+(y_1-y_0)^2}$


Usen las funciones estandar de numpy `sqrt` y `**` para calcularla. PpPpueden usar el atributo `.values` para restar varias columnas entre si. Como adicional vean como pueden calcularla usando la norma del vector restado de las dos narices: [np.linalg.norm](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html).

In [ ]:
# COMPLETAR: Crear una nueva columna que mida la distancia entre las narices de los ratones
df_filter["distance_nose"] = ...


d) Finalmente graficar en un violinplot la distribución de la columna nueva para cada comportamiento registrado.

In [ ]:
# COMPLETAR: Graficar para cada comportamiento registrado la distribución de estas distancias (usar un violinplot)
...

## 1.3 Correlaciones de movimiento

Por último podemos ver que sucede con la correlación entre las columnas dependiendo de cada comportamiento. Para eso vamos a seleccionar las columnas asociadas a cada punto del cuerpo en el dataframe filtrado y crear nuevas columnas distancia entre ellos y ver si existe correlaciones entre ellas durante la etiqueta `mount`.

In [ ]:
# Vamos a seleccionar las columnas y crear las de distancia 
body_parts    = ["nose", "left_ear", "right_ear", "neck", "left_hip", "right_hip", "tail_base"]
cols_intruder = [f"keypoints_mousse_intruder_{coord}_"+part for coord in ["x","y"] for part in body_parts ] # esto es lo que se conoce como lista por comprehension
cols_resident = [f"keypoints_mousse_resident_{coord}_"+part for coord in ["x","y"] for part in body_parts ]
# Hacemos la diferencia coordenada a coordenada
diff_array    = df_filter[cols_intruder].values-df_filter[cols_resident].values
print(diff_array.shape)

# Diff ahora tiene las diferencias coordenada a coordenada, nos falta recorrer las partes del cuerpo y calcular la norma vectorial para obtener las distnacias
for i, part in enumerate(body_parts): # enumerate nos devuelve una tupla (i, part) donde el i es la enumeración de los objetos que hay dentro de la lista
    df_filter[f"distance_{part}"] = np.linalg.norm(diff_array[:,[i,i+7]],axis=1) # 7 porque las columnas son las 7 partes del cuerpo

Completar la siguiente celda para graficar el mapa de calor de las correlaciones:

In [ ]:
# COMPLETAR
cols     = [f"distance_{part}" for part in body_parts]
corr_mat = ...

# Plotear un heatmap de las correlaciones
...

# (*) Ejercicio 2: Utilizando las features!

Las features constituyen un vector de 60 dimensiones que funcionan como descriptores de cada uno de los frames de los videos de los ratones. Como tienen 60 dimensiones es muy dificil graficar algo, por lo que se hace necesario recurrir a técnicas de [reducción dimensional](https://stackabuse.com/dimensionality-reduction-in-python-with-scikit-learn/). Estas técnicas nos permiten reducir la cantidad de datos preservando caracteristicas de lo datos originales. Para ejempliflicar esto utilizaremos una técnica que se conoce como PCA o [Principal Component Analysis](https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html) que busca la direcciones que maximizan la variabilidad de los datos. Veamos un ejemplo.

In [ ]:
# Veamos que pasa cuando observamos las features de los frames si nos permiten separar los comportamientos
# Para eso creamos esta función auxiliar que nos devuelve un dataframe con las 60 features mas las anotaciones
def load_annotator_features(feat_dict, annotator):
    
    dict_aux = feat_dict[annotator]
    annot_keys = {0:'attack',1: 'investigation',2: 'mount',3:'other'}
    
    df = pd.DataFrame(dict_aux['features'] , dtype=float)
    df['annotations'] = dict_aux['annotations']
    df['annotations'] = df['annotations'].map(annot_keys)
    
    return df
  
df_features = load_annotator_features(training_features, 'task1/train/mouse001_task1_annotator1')
df_features.head()

a) Para hacer PCA vamos a necesitar reescalar las columnas de tal manera que queden estandarizadas, pero para eso necesitamos seleccionar las columnas y llevaras a Numpy.

In [ ]:
# COMPLETAR: seleccionar las columnas correspondientes a los features y guardarla en una variable X como numpy array 
X = ...
print(X.shape)

In [ ]:
# Para que PCA funcione correctamente debemos estandarizar los datos (media 0, desvio 1) y para eso usaremos una funcion que nos provee Scikit Learn
scaler = StandardScaler()
# Creamos un array nuevo que estará escalado
X_scaled = scaler.fit_transform(X)
# Verifiquemos (ojo que la media puede no dar exactamente 0 pero si un numero muy pequeño)
#print(X_scaled.mean(axis=0))
#print(X_scaled.std(axis=0))

In [ ]:
# Estamos listos para reducir la dimension y para eso tenemos que crear nuestro objeto que va a reducir la dimension PCA
# Indicandole el número de dimensiones con el cual nos queremos quedar
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Creamos un dataframe
principal_df = pd.DataFrame(data = X_pca, columns = ['PC1', 'PC2'])
principal_df['annotations'] = df_features['annotations']
print(principal_df.shape)
principal_df.head()

In [ ]:
# Grafiquemos que sucede con la distribución espacial de las features - puede tardar un poco
sns.jointplot(x=principal_df.PC1, y=principal_df.PC2, cmap="Blues", shade=True, kind='kde');

b) Graficar un scatterplot de las dos componentes asociadas a cada frame agrupando los datos por anotación. ¿Que conclusiones puede sacar?

In [ ]:
# COMPLETAR
...

b) Realizar una reducción a 3 dimensiones a los mismos datos y graficarlos utilizando la función provista por la librería [Plotly](https://plotly.com/python/).

In [ ]:
# PCA
pca = ...
components = ...

principal_df = ...
principal_df['annotations'] = df_features['annotations']

# 3D scatterplot interactivo
fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=principal_df['annotations'], size=0.1*np.ones(len(X)), opacity = 1,
    title='PCA plot in 3D',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
    width=650, height=500
)
fig.show()

¿Ve una mejor separación entre los frames de asociados a cada comportamiento?

# Adicionales: Animaciones de los 🐀

Esta celda contiene algunas funciones auxiliares que usaremos para crear una animación de los movimientos del ratón. Puedne ignorar el contenido, pero asegúrese de ejecutarlo o la siguiente sección no funcionará. Esta parte es solo para ilustrar y motivar todas las cosas que podemos hacer con estos datos y python!

In [ ]:
#@title Funciones auxiliares para los gifs y el raster
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import colors
from matplotlib import rc
import matplotlib.patches as mpatches

rc('animation', html='jshtml')

# Note: Image processing may be slow if too many frames are animated.

# Plotting constants
FRAME_WIDTH_TOP = 1024
FRAME_HEIGHT_TOP = 570

RESIDENT_COLOR = 'lawngreen'
INTRUDER_COLOR = 'skyblue'

PLOT_MOUSE_START_END = [(0, 1), (0, 2), (1, 3), (2, 3), (3, 4),
                        (3, 5), (4, 6), (5, 6), (1, 2)]
class_to_color = {'other': 'white', 'attack' : 'red', 'mount' : 'green',
                  'investigation': 'orange'}
class_to_number = {s: i for i, s in enumerate(vocab)}
number_to_class = {i: s for i, s in enumerate(vocab)}


def num_to_text(anno_list):
  return np.vectorize(number_to_class.get)(anno_list)


def set_figax():
  fig = plt.figure(figsize=(6, 4))

  img = np.zeros((FRAME_HEIGHT_TOP, FRAME_WIDTH_TOP, 3))

  ax = fig.add_subplot(111)
  ax.imshow(img)

  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  return fig, ax


def plot_mouse(ax, pose, color):
  # Draw each keypoint
  for j in range(7):
    ax.plot(pose[j, 0], pose[j, 1], 'o', color=color, markersize=5)

  # Draw a line for each point pair to form the shape of the mouse

  for pair in PLOT_MOUSE_START_END:
    line_to_plot = pose[pair, :]
    ax.plot(line_to_plot[:, 0], line_to_plot[
            :, 1], color=color, linewidth=1)


def animate_pose_sequence(video_name, keypoint_sequence, start_frame = 0, stop_frame = 100,
                          annotation_sequence = None):
  # Returns the animation of the keypoint sequence between start frame
  # and stop frame. Optionally can display annotations.
  seq = keypoint_sequence.transpose((0,1,3,2))

  image_list = []

  counter = 0
  for j in range(start_frame, stop_frame):
    if counter%20 == 0:
      print("Processing frame ", j)
    fig, ax = set_figax()
    plot_mouse(ax, seq[j, 0, :, :], color=RESIDENT_COLOR)
    plot_mouse(ax, seq[j, 1, :, :], color=INTRUDER_COLOR)

    if annotation_sequence is not None:
      annot = annotation_sequence[j]
      annot = number_to_class[annot]
      plt.text(50, -20, annot, fontsize=16,
               bbox=dict(facecolor=class_to_color[annot], alpha=0.5))

    ax.set_title(
        video_name + '\n frame {:03d}.png'.format(j))

    ax.axis('off')
    fig.tight_layout(pad=0)
    ax.margins(0)

    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(),
                                    dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(
        fig.canvas.get_width_height()[::-1] + (3,))

    image_list.append(image_from_plot)

    plt.close()
    counter = counter + 1

  # Plot animation.
  fig = plt.figure()
  plt.axis('off')
  im = plt.imshow(image_list[0])

  def animate(k):
      im.set_array(image_list[k])
      return im,
  ani = animation.FuncAnimation(fig, animate, frames=len(image_list), blit=True)
  return ani


def plot_behavior_raster(annotation_sequence, start_frame=0,
                         stop_frame=100,
                         title="Behavior Labels"):
  # Plot annotations as a behavior raster

  # Map annotations to a number.
  annotation_num = []
  for item in annotation_sequence[start_frame:stop_frame]:
    annotation_num.append(class_to_number[item])

  all_classes = list(set(annotation_sequence[start_frame:stop_frame]))

  cmap = colors.ListedColormap(['red', 'orange', 'green', 'white'])
  bounds=[-0.5, 0.5, 1.5, 2.5, 3.5]
  norm = colors.BoundaryNorm(bounds, cmap.N)

  height = 200
  arr_to_plot = np.repeat(np.array(annotation_num)[:, np.newaxis].transpose(),
                                                  height, axis = 0)

  fig, ax = plt.subplots(figsize = (16, 3))
  ax.imshow(arr_to_plot, interpolation='none',cmap=cmap, norm=norm)

  ax.set_yticks([])
  ax.set_xlabel('Frame Number')
  plt.title(title)

  legend_patches = []
  for item in all_classes:
    legend_patches.append(mpatches.Patch(color=class_to_color[item], label=item))

  plt.legend(handles=legend_patches,loc='center left', bbox_to_anchor=(1, 0.5))

  plt.tight_layout()

## Visualizando los mivimientos 🎥

¡Hagamos algunos gifs de nuestra secuencia de muestra para tener una idea de cómo se ven los datos sin procesar! Puede cambiar los valores de `start_frame` y `stop_frame` para mirar alrededor. Esto va a demorar un poco.

In [ ]:
keypoint_sequence = single_sequence['keypoints']
annotation_sequence = single_sequence['annotations']

ani = animate_pose_sequence(sample_sequence_key,
                            keypoint_sequence,
                            start_frame=5000,
                            stop_frame=5100,
                            annotation_sequence=annotation_sequence)

# Display the animation on colab
ani

## Behavior Raster

También podemos ver un **Behavior Raster**, que muestra qué comportamiento se anotó en cada cuadro de este video.

In [ ]:
annotation_sequence = single_sequence['annotations']
text_sequence = num_to_text(annotation_sequence)
sns.set(style="ticks")
plot_behavior_raster(
    text_sequence,
    start_frame=0,
    stop_frame=len(annotation_sequence)
    )